In [1]:
from ipynb.fs.full.PaiGowEvaluator_NoJoker import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
suits = ['♠', '♦', '♥', '♣']
cards = [(x,y) for x in hierarchy_values for y in suits]
random.shuffle(cards)

def deal(cards):
    dealt_cards = []
    for i in range(7):
        dealt_cards.append(cards.pop(0))
    return dealt_cards



In [3]:
def print_cards(cards):
    values = []
    suits = []
    for x, y in cards:
        values.append(x)
        suits.append(y)

        
    lines = [[] for i in range(9)]   
    for i in range(len(cards)):    
        
        lines[0].append('┌─────────┐')
        lines[1].append('│{}        │'.format(values[i]))  # use two {} one for char, one for space or char
        lines[2].append('│         │')
        lines[3].append('│         │')
        lines[4].append('│    {}    │'.format(suits[i]))
        lines[5].append('│         │')
        lines[6].append('│         │')
        lines[7].append('│       {} │'.format(values[i]))
        lines[8].append('└─────────┘')
        
    
    
    result = []
    for index, line in enumerate(lines):
        print(''.join(lines[index]))
    

In [4]:
def compare_highs(a,b):
    a = sorted(a, reverse = True)
    b = sorted(b, reverse = True)
    for i in range(len(a)):
        if a[i] > b[i]:
            return 1
        elif a[i] < b[i]:
            return 0
        else:
            pass
        if i == len(a)-1:
            return 1 

In [5]:
#hand 1 is dealer
#hand 2 is player
# 1 entails a win for the dealer 
def compare_hands_of_five(hand1, hand2):
    
    score1 = evaluate_bottom(hand1)
    score2 = evaluate_bottom(hand2)
    #print(score1, score2)
    if score1 > score2:
        return 1
    elif score1 < score2:
        return 0
    else:
        
        values1 = []
        values2 = []

        five_hand1 = sorted(hand1, key=lambda x: hierarchy_values.index(x[0]), reverse = True)
        five_hand2 = sorted(hand2, key=lambda x: hierarchy_values.index(x[0]), reverse = True)

        for x,y in five_hand1:
            values1.append(x)
        for x,y in five_hand2:
            values2.append(x)
            
        ## HIGH CARD
        if score1 == 1:

            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            return compare_highs(values1, values2)
        
        
        ## ONE PAIR
        elif score1 == 2:
            
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            
            value_counts1 = Counter(values1).most_common()
            value_counts2 = Counter(values2).most_common() 
            

            
            pairs1 = []
            singles1 = []
            
            for x, y in value_counts1:
                if y == 1:
                    singles1.append(x)
            
                singles1 = sorted(singles1, reverse = True)
                if y == 2:
                    pairs1.append(x)
                    
            pairs2 = []
            singles2 = []
            
            for x, y in value_counts2:
                if y == 1:
                    singles2.append(x)
                singles2 = sorted(singles2, reverse = True)
                if y == 2:
                    pairs2.append(x)
            
            #first compare pairs
            if pairs1[0] > pairs2[0]:
                return 1
            elif pairs1[0] < pairs2[0]:
                return 0
            elif pairs1[0] == pairs2[0]:
                return compare_highs(singles1, singles2)
        
        ##TWO PAIR
        elif score1 == 3:
            
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            
            value_counts1 = Counter(values1).most_common()
            value_counts2 = Counter(values2).most_common() 
            

            #create pairs and singles for each hand
            pairs1 = []
            singles1 = []
            
            for x, y in value_counts1:
                if y == 1:
                    singles1.append(x)
                singles1 = sorted(singles1,  reverse = True)
                
                if y == 2:
                    pairs1.append(x)
                pairs1 = sorted(pairs1, reverse = True)
                    
            pairs2 = []
            singles2 = []
            
            for x, y in value_counts2:
                if y == 1:
                    singles2.append(x)
                singles2 = sorted(singles2, reverse = True)
                
                if y == 2:
                    pairs2.append(x)
                pairs2 = sorted(pairs2,  reverse = True)
            
            if pairs1[0] > pairs2[0]:
                return 1
            elif pairs1[0] < pairs2[0]:
                return 0
            elif pairs1[0] == pairs2[0]:
                if pairs1[1] > pairs2[1]:
                    return 1
                elif pairs1[1] < pairs2[1]:
                    return 0
                elif pairs1[1] == pairs2[1]:
                    return compare_highs(singles1, singles2)
                
        # TRIPS OR FULL HOUSE
        elif score1 == 4 or score1 == 7:
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            
            value_counts1 = Counter(values1).most_common()
            value_counts2 = Counter(values2).most_common() 
            

            #create trips for each hand
            trips1 = []
            
            for x, y in value_counts1:
               
                if y == 3:
                    trips1.append(x)
      
            trips2 = []
            
            for x, y in value_counts2:
                if y == 3:
                    trips2.append(x)

            #compare trips 
            if trips1[0] > trips2[0]:
                return 1
            elif trips1[0] < trips2[0]:
                return 0

            
        ## STRAIGHT OR STRAIGHT FLUSH   
        elif score1 == 5 or score1 == 9:
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            best_straight = [10, 11, 12, 13, 14]
            second_best_straight = [1, 2, 3, 4, 5]
            
            result_high_1 =  all(elem in values1 for elem in best_straight)
            result_low_1 =  all(elem in values2 for elem in second_best_straight)

            result_high_2 =  all(elem in values1 for elem in best_straight)
            result_low_2 =  all(elem in values2 for elem in second_best_straight)
            
            if result_high_1 == True:
                return 1
            elif result_high_2 == True:
                return 0
            elif result_low_1 == True:
                return 1
            elif result_low_2 == True:
                return 0
            else:
                values1 = sorted(values1, reverse = True)
                values2 = sorted(values2, reverse = True)
                return compare_highs(values1, values2)
            
        ## FLUSH
        elif score1 == 6:
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            values1 = sorted(values1, reverse = True)
            values2 = sorted(values2, reverse = True)
            return compare_highs(values1, values2)
        
        ## FOUR OF A KIND
        elif score1 == 8:
            values1 = [translate(x) for x in values1]
            values2 = [translate(x) for x in values2]
            
            value_counts1 = Counter(values1).most_common()
            value_counts2 = Counter(values2).most_common() 
            
            #create fours for each hand
            fours1 = []

            for x, y in value_counts1:
                if y == 4:
                    fours1.append(x)
                
            fours2 = []
            
            for x, y in value_counts2:
                if y == 4:
                    fours2.append(x)

            #compare fours 
            if fours1[0] > fours2[0]:
                return 1
            elif fours1[0] < fours2[0]:
                return 0
            
            
        ### ROYAL FLUSH DEALER ALWAYS WINS
        elif score1 == 10:
            return 1
            
        

            
                




0

In [6]:
def compare_hands_of_two(hand1, hand2):
    values1, values2 = [], []
    for x, y in hand1:
        values1.append(x)
    values1 = [translate(x) for x in values1]

    for x, y in hand2:
        values2.append(x)
    values2 = [translate(x) for x in values2]
    #print(values1, values2)
    #set of two pairs    
    if values1[0] == values1[1] and values2[0] == values2[1]:
        if values1[0] >= values2[0]:
            return 1
        else:
            return 0
    #hand1 has pair and hand 2 doesn't
    if values1[0] == values1[1] and values2[0] != values2[1]:
        return 1
    #hand2 has pair and hand1 doesn't
    if values1[0] != values1[1] and values2[0] == values2[1]:
        return 0
    #no pairs
    if values1[0] != values1[1] and values2[0] != values2[1]:
        high1, high2 = max(values1), max(values2)
        if high1 > high2:
            return 1
        elif high2 > high1:
            return 0
        else:
            min1, min2 = min(values1), min(values2)
            if min1 >= min2:
                return 1
            else:
                return 0

In [7]:
def five_better_than_two(five, two):
    
    values1, values2 = [], []
    for x, y in five:
        values1.append(x)
    values1 = [translate(x) for x in values1]

    for x, y in two:
        values2.append(x)
    values2 = [translate(x) for x in values2]
    
    if evaluate_bottom(five) <=2:
        
        #if pair in two hand
        if values2[0] == values2[1]:
            pairs = []
            value_counts1 = Counter(values1).most_common()
            for x,y in value_counts1:
                if y == 2:
                    pairs.append(x)
            #if only one pair
            if len(pairs) == 1:
                #if five card hand has a better pair than two card hand
                if pairs[0] > values2[0]:
                    return True
                else:
                    return False
            else:
                return False
   #BOTH HAVE HIGH CARDS 
   # NEED TO MAKE IT SO WE HAVE PAIR VS NOT PAIR
   # IF NO PAIR< THEN COMPARE HIGH CARD
        else:
            if evaluate_bottom(five) == 2:
                return True
            elif evaluate_bottom(five) == 1:
                if max(values1) > max(values2):
                    return True
                elif max(values1) < max(values2):
                    return False
                elif max(values1) == max(values2):
                    if sorted(values1, reverse = True)[1] >= min(values2):
                        return True
                    elif sorted(values1, reverse = True)[1] < min(values2):
                        return False

    else:
         return True

In [58]:
# NOW, WE OPTIMIZE PLAYER'S PLAY AGAINST DEALER


def face_dealer(dealer_5, dealer_2, player_hand):
    player_hands = list(itertools.combinations(player_hand, 5))
    
    max_score = 0
    best_hand = 0
    for i in range(len(player_hands)):
        set_of_two = tuple(set(player_hand).difference(set(player_hands[i])))
        if five_better_than_two(player_hands[i], set_of_two):
            #print(player_hands[i], set_of_two)
            #print(dealer_5)
            five_score = compare_hands_of_five(dealer_5, player_hands[i])
            two_score = compare_hands_of_two(dealer_2, tuple(set(player_hand).difference(set(player_hands[i]))))
            #print("Five score is " + str(five_score))
            #print("Two score is " + str(two_score))
            # I did my scoring system in terms of the dealer beating the player before
            #if we make this variable 2-score, then it would be the player vs dealer
            # ie, if the dealer had a 2, then the player is 2-2 = 0. if dealer had 0, player is 2-0 = 2
            player_score = 2 - (five_score + two_score)

            if player_score > max_score:
                max_score = player_score
                best_hand = i
#     max_score = 0
    return max_score
#, player_hands[best_hand], tuple(set(player_hand).difference(set(player_hands[best_hand])))
    
        

In [32]:
def best_hand_against_dealer(dealer_5, dealer_2, player_hand):
        player_hands = list(itertools.combinations(player_hand, 5))

        max_score = 0
        best_hand = 0
        for i in range(len(player_hands)):
            set_of_two = tuple(set(player_hand).difference(set(player_hands[i])))
            if five_better_than_two(player_hands[i], set_of_two):
                #print(player_hands[i], set_of_two)
                #print(dealer_5)
                five_score = compare_hands_of_five(dealer_5, player_hands[i])
                two_score = compare_hands_of_two(dealer_2, tuple(set(player_hand).difference(set(player_hands[i]))))
                player_score = 2 - (five_score + two_score)

                if player_score > max_score:
                    max_score = player_score
                    best_hand = i
        best_5 = tuple(player_hands[best_hand])
        best_2 = tuple(set(player_hand).difference(set(player_hands[best_hand])))
    #     max_score = 0
        return best_5, best_2
    
    


In [12]:

# print(dealer_hand)
# dealer_5 = evaluate_seven(dealer_hand)[0]
# dealer_2 = evaluate_seven(dealer_hand)[1]
# casino_win = 0
# for i in range(6):
#     score = face_dealer(dealer_5, dealer_2, players[i])
#     if score == 0:
#         casino_win +=1
#     if score == 2:
#         casino_win -=1
# casino_win


[(3, '♣'), ('J', '♠'), (9, '♥'), (7, '♥'), (9, '♠'), (4, '♣'), ('Q', '♦')]


-5

In [ ]:
def play_game(buy_in = 100, play = True):
    while play == True:
        # create hand
        hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
        suits = ['♠', '♦', '♥', '♣']
        cards = [(x,y) for x in hierarchy_values for y in suits]
        random.shuffle(cards)
        
        #deal hand
        dealer_hand = deal(cards)
        players = []
        for i in range(6):
            players.append(deal(cards))
            
        #simulate hand
        
        
        
        

In [57]:
#GET PAYOUTS


hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
suits = ['♠', '♦', '♥', '♣']
cards = [(x,y) for x in hierarchy_values for y in suits]
dealer_hand = deal(cards)

def get_payouts(dealer_hand):
    hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
    suits = ['♠', '♦', '♥', '♣']
    cards = [(x,y) for x in hierarchy_values for y in suits]
    cards = deal(cards)


    dealer_5 = evaluate_seven(dealer_hand)[0]
    dealer_2 = evaluate_seven(dealer_hand)[1]
    #print(dealer_5, dealer_2)

    trials = 250
    win_list = []
    for i in range(trials):
        cards = [(x,y) for x in hierarchy_values for y in suits]
        rest_of_cards = list(set(cards).difference(set(dealer_hand)))
        random.shuffle(rest_of_cards)
        casino_win = 0
        players =[]
        for j in range(6):
            players.append(rest_of_cards[7*j: 7 * (j+1)])
            score = face_dealer(dealer_5, dealer_2, players[j])
            if score == 0:
                casino_win +=1
            if score == 2:
                casino_win -=1
        win_list.append(casino_win)

    win_distr = Counter(win_list).most_common() 



    payouts = []
    for i in range(-6,7):
        if i in [win for win, freq in win_distr]:
            ##FIX THIS MAKE THIS OVER TRIALS
            if dict(win_distr)[i]/trials > 0.05:
                payouts.append(round(5/((dict(win_distr)[i]/trials) + 0.3)))
            else:
                payouts.append(100)

        else:
            payouts.append(100)
    return [list(range(-6,7)), payouts]



In [59]:
#GET PAYOUTS




def show_distribution(dealer_hand):
    hierarchy_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']
    suits = ['♠', '♦', '♥', '♣']
    cards = [(x,y) for x in hierarchy_values for y in suits]
    cards = deal(cards)


    dealer_5 = evaluate_seven(dealer_hand)[0]
    dealer_2 = evaluate_seven(dealer_hand)[1]
    print(dealer_5, dealer_2)

    trials = 1000
    win_list = []
    for i in range(trials):
        cards = [(x,y) for x in hierarchy_values for y in suits]
        rest_of_cards = list(set(cards).difference(set(dealer_hand)))
        random.shuffle(rest_of_cards)
        casino_win = 0
        for j in range(6):
            players[j] = rest_of_cards[7*j: 7 * (j+1)]
            score = face_dealer(dealer_5, dealer_2, players[j])
            if score == 0:
                casino_win +=1
            if score == 2:
                casino_win -=1
        win_list.append(casino_win)



    print_cards(evaluate_seven(dealer_hand)[1])
    print_cards(evaluate_seven(dealer_hand)[0])
    values, bins, bars = plt.hist(win_list, bins=np.arange(-7, 8), rwidth = 0.8)
    plt.xlabel("Dealer wins per deck dealing")
    plt.ylabel("Frequency (out of " + str(trials))
    plt.title = ('Distribution of Wins')
    plt.bar_label(bars, fontsize=20, color='navy')
    plt.margins(x=0.01, y=0.1)
    plt.show()



